# VNPY 回測模組

## 引入所需模組包

In [1]:
from vnpy.app.cta_strategy.backtesting import BacktestingEngine, OptimizationSetting
from vnpy.app.cta_strategy.base import BacktestingMode   #backtesting 
from datetime import datetime    #time
from vnpy.trader.constant import Interval, Exchange
# from vnpy.app.atr_rsi_strategy import AtrRsiStrategy

from vnpy.app.portfolio_strategy.backtesting import BacktestingEngine as PairBacktestEngine

import matplotlib
%matplotlib inline

## 交易成本設定

In [2]:
import re


def getCommission(symbol,
                  exchange,
                  cost,
                  multiplier,
                  qty,
                  Real=False,
                  direction=''):
    
    """計算個別部位的單邊交易成本"""
    try:
        if exchange == 'TFE':
            if re.match('MTX[0-9]*', symbol):  # 小台指期
                return 50 + (1 * 50 + cost * (2 / 100000) * multiplier
                             ) * qty if not Real else 50 + (cost *
                                                            (2 / 100000) *
                                                            multiplier) * qty
            elif re.match('TX[0-9]*',
                          symbol) or re.match('TE[0-9]*', symbol) or re.match(
                              'TF[0-9]*', symbol):  # 大台指期
                return 50 + (1 * 200 + cost * (2 / 100000) * multiplier
                             ) * qty if not Real else 50 + (cost *
                                                            (2 / 100000) *
                                                            multiplier) * qty
            elif re.match(r"TX[O0-9][0-9]*[A-Z][0-9]", symbol):  # 台指選
                return 30 + (getMinimumTickOpt(cost) + cost * 2 /
                             100000) * qty * multiplier if not Real else 30 + (
                                 cost * 2 / 100000) * qty * multiplier
            elif re.match(r"[A-Z][A-Z]F[0-9][0-9]", symbol):  # 個股期貨
                return 50 + (
                    getMinimumTick(cost) + cost *
                    (2 / 100000)) * qty * multiplier if not Real else 50 + (
                        cost * (2 / 100000)) * qty * multiplier
        elif exchange == 'TSE':
            tick = getMinimumTick(cost)
            commission = cost * (0.1425 / 100) * multiplier
            commission = 20 if commission < 20 else commission
            fee = cost * (0.3 / 100) * multiplier
            slide = tick * multiplier
            if direction == 'EXIT' or direction == 0:
                return (commission + fee +
                        slide) * qty if not Real else (commission + fee) * qty
            else:
                return (commission +
                        slide) * qty if not Real else commission * qty
    except Exception as e:
        print(e)


def getMinimumTickOpt(cost):
    if cost < 10:
        return 0.1
    elif cost < 50:
        return 0.5
    elif cost < 500:
        return 1
    elif cost < 1000:
        return 5
    else:
        return 10


def getMinimumTick(cost):
    if cost < 10:
        return 0.01
    elif cost < 50:
        return 0.05
    elif cost < 100:
        return 0.1
    elif cost < 500:
        return 0.5
    elif cost < 1000:
        return 1
    else:
        return 5

## 布林通道逆勢策略 + 固定停損停利

In [38]:
from vnpy.app.cta_strategy import (
    CtaTemplate,
    StopOrder,
    TickData,
    BarData,
    TradeData,
    OrderData,
    BarGenerator,
    ArrayManager,
)


class BollChannelStrategy(CtaTemplate):
    """"""

    author = "Xi Qi Capital"

    boll_window = 20
    boll_dev = 2
    sl_multiplier = 0.5    #stop losee multiplier
    tp_multiplier = 2      
    sl_pct = 0.1          #stop loss percentage
    tp_pct = 0.3
    fixed_size = 1

    boll_up = 0
    boll_down = 0
 
    long_stop = 0
    long_profit = 0
    short_stop = 0
    short_profit = 0
    
    

    parameters = [
        "boll_window",
        "boll_dev",
        "tp_multiplier",
        "sl_multiplier",
        "fixed_size"
    ]
    variables = [
        "boll_up",
        "boll_down",        
        "long_stop",
        "short_stop",
        "long_profit",
        "short_profit",
    ]

    def __init__(self, cta_engine, strategy_name, vt_symbol, setting):
        """"""
        super().__init__(cta_engine, strategy_name, vt_symbol, setting)

        self.bg = BarGenerator(self.on_bar)
        self.am = ArrayManager()

    def on_init(self):
        """
        Callback when strategy is inited.
        """
        self.write_log("策略初始化")
        self.load_bar(10)

    def on_start(self):
        """
        Callback when strategy is started.
        """
        self.write_log("策略启动")

    def on_stop(self):
        """
        Callback when strategy is stopped.
        """
        self.write_log("策略停止")

    def on_tick(self, tick: TickData):
        """
        Callback of new tick data update.
        """
        self.bg.update_tick(tick)


    def on_bar(self, bar: BarData):
        """"""
        self.cancel_all()                  #確認沒有其他order單                                   

        am = self.am
        am.update_bar(bar)                 #arrary manager更新最新bar
        if not am.inited:
            return
        
        
        self.close_price = am.close_array[-1] 
        self.pre_close_price = am.close_array[-2]
        
        self.high_price = am.high_array[-1]
        self.pre_high_price = am.high_array[-2]
       
        self.low_price = am.low_array[-1]
        self.pre_low_price = am.low_array[-2]
        
        
        self.boll_up, self.boll_down = am.boll(self.boll_window, self.boll_dev)    #array manager內有布林通道function，輸入參數即可
       
                 

        if self.pos == 0:                        #建倉
              
            if self.close_price >= self.pre_close_price and self.pre_high_price >= self.boll_up:  
                self.short(self.boll_up, self.fixed_size, True)
                self.short_entry_price = self.boll_up
                
            elif self.close_price <= self.pre_close_price and self.pre_low_price <= self.boll_down: 
                self.buy(self.boll_down, self.fixed_size, True)
                self.buy_entry_price = self.boll_down
            

        elif self.pos > 0:  
            
            
            self.long_stop = self.buy_entry_price * (1- self.sl_pct)   #stop loss percentage
            self.long_profit = self.buy_entry_price * (1 + self.tp_pct)    #take profit percentage
          
            
                            
            if self.close_price <= self.long_stop:
                 self.sell(self.long_stop, abs(self.pos), True)
                
            elif self.close_price >= self.long_profit:
                 self.sell(self.long_profit, abs(self.pos), True)
                
            
        elif self.pos < 0:
            
            self.short_stop = self.short_entry_price * (1+ self.sl_pct)   #stop loss percentage
            self.short_profit = self.short_entry_price * (1 - self.tp_pct)    #take profit percentage
            
            if self.close_price >= self.long_stop:
                 self.cover(self.long_stop, abs(self.pos), True)
                
            elif self.close_price <= self.long_profit:
                 self.cover(self.long_profit, abs(self.pos), True)
            

        self.put_event()

    def on_order(self, order: OrderData):
        """
        Callback of new order data update.
        """
        pass

    def on_trade(self, trade: TradeData):
        """
        Callback of new trade data update.
        """
        self.put_event()

    def on_stop_order(self, stop_order: StopOrder):
        """
        Callback of stop order update.
        """
        pass


## 回測結果

In [ ]:
from functools import partial

tickers = '2330.TSE'
symbol, exchange = tickers.split('.')
get_commission = partial(getCommission, symbol=symbol, exchange=exchange)
engine = BacktestingEngine()
engine.set_parameters(
    vt_symbol=tickers,
    interval=Interval.DAILY,
    start=datetime(2010, 12, 1),
    end=datetime(2019, 12, 1),
    rate=get_commission,
    slippage=getMinimumTick,
    size=1000,
    pricetick=getMinimumTick,
    capital=100_000,
    mode=BacktestingMode.BAR,
    collection_name='TSE')

boll_setting = {
    'boll_window' : 20,
    'boll_dev' : 2,
    'sl_pct': 0.1,
    'tp_pct': 0.3,
    'fixed_size' : 1.5,
}
engine.add_strategy(BollChannelStrategy, boll_setting)
engine.load_data()
engine.run_backtesting()
df = engine.calculate_result()
engine.calculate_statistics()
engine.show_chart()


2020-12-04 18:34:55.081994	开始加载历史数据
2020-12-04 18:34:55.081994	加载进度： [1%]
2020-12-04 18:34:55.081994	加载进度： [2%]
2020-12-04 18:34:55.081994	加载进度： [3%]
2020-12-04 18:34:55.081994	加载进度： [4%]
2020-12-04 18:34:55.081994	加载进度： [5%]
2020-12-04 18:34:55.081994	加载进度： [5%]
2020-12-04 18:34:55.081994	加载进度： [6%]
2020-12-04 18:34:55.081994	加载进度： [7%]
2020-12-04 18:34:55.082987	加载进度： [8%]
2020-12-04 18:34:55.082987	加载进度： [9%]
2020-12-04 18:34:55.082987	加载进度：# [10%]
2020-12-04 18:34:55.082987	加载进度：# [11%]
2020-12-04 18:34:55.082987	加载进度：# [12%]
2020-12-04 18:34:55.082987	加载进度：# [13%]
2020-12-04 18:34:55.082987	加载进度：# [14%]
2020-12-04 18:34:55.082987	加载进度：# [15%]
2020-12-04 18:34:55.082987	加载进度：# [16%]
2020-12-04 18:34:55.082987	加载进度：# [16%]
2020-12-04 18:34:55.082987	加载进度：# [17%]
2020-12-04 18:34:55.082987	加载进度：# [18%]
2020-12-04 18:34:55.082987	加载进度：# [19%]
2020-12-04 18:34:55.082987	加载进度：## [20%]
2020-12-04 18:34:55.082987	加载进度：## [21%]
2020-12-04 18:34:55.082987	加载进度：## [22%]
2020-12-04 18:34:55.0

## 移動停損停利


In [12]:
class BollChannelStrategy(CtaTemplate):
    """"""

    author = "Xi Qi Capital"

    boll_window = 20
    boll_dev = 2
    sl_multiplier = 0.5    #stop losee multiplier
    tp_multiplier = 2      
    sl_pct = 0.03          #stop loss percentage
    tp_pct = 0.1
    fixed_size = 1

    boll_up = 0
    boll_down = 0
 
    long_stop = 0
    long_profit = 0
    short_stop = 0
    short_profit = 0
    
    

    parameters = [
        "boll_window",
        "boll_dev",
        "tp_multiplier",
        "sl_multiplier",
        "fixed_size"
    ]
    variables = [
        "boll_up",
        "boll_down",        
        "long_stop",
        "short_stop",
        "long_profit",
        "short_profit",
    ]

    def __init__(self, cta_engine, strategy_name, vt_symbol, setting):
        """"""
        super().__init__(cta_engine, strategy_name, vt_symbol, setting)

        self.bg = BarGenerator(self.on_bar)
        self.am = ArrayManager()

    def on_init(self):
        """
        Callback when strategy is inited.
        """
        self.write_log("策略初始化")
        self.load_bar(10)

    def on_start(self):
        """
        Callback when strategy is started.
        """
        self.write_log("策略启动")

    def on_stop(self):
        """
        Callback when strategy is stopped.
        """
        self.write_log("策略停止")

    def on_tick(self, tick: TickData):
        """
        Callback of new tick data update.
        """
        self.bg.update_tick(tick)

    def on_bar(self, bar: BarData):
        """"""
        self.cancel_all()                  #確認沒有其他order單                                   

        am = self.am
        am.update_bar(bar)                 #arrary manager更新最新bar
        if not am.inited:
            return
        
        
        self.close_price = am.close_array[-1] 
        self.pre_close_price = am.close_array[-2]
        
        self.high_price = am.high_array[-1]
        self.pre_high_price = am.high_array[-2]
        
        self.low_price = am.low_array[-1]
        self.pre_low_price = am.low_array[-2]
        
        
        self.boll_up, self.boll_down = am.boll(self.boll_window, self.boll_dev)    #array manager內有布林通道function，輸入參數即可
        self.sl_dev = self.boll_dev * self.sl_multiplier
        self.tp_dev = self.boll_dev * self.tp_multiplier
        
        self.sl_up, self.sl_down = am.boll(self.boll_window, self.sl_dev)
        self.tp_up, self.tp_down = am.boll(self.boll_window, self.tp_dev)
        
        if self.pos == 0:                        #建倉
              
            if self.close_price >= self.pre_close_price and self.pre_high_price >= self.boll_up:  
                self.short(self.boll_up, self.fixed_size, True)
                self.entry_price = self.boll_up
                
            elif self.close_price <= self.pre_close_price and self.pre_low_price <= self.boll_down: 
                self.buy(self.boll_down, self.fixed_size, True)
                self.entry_price = self.boll_down
            

        elif self.pos > 0:  
            
            
            self.long_stop = self.sl_down   
            self.long_profit = self.tp_up 
          
            
                            
            if self.close_price <= self.long_stop:
                 self.sell(self.long_stop, abs(self.pos), True)
                
            elif self.close_price >= self.long_profit:
                 self.sell(self.long_profit, abs(self.pos), True)
                
            
        elif self.pos < 0:
            
            self.short_stop = self.sl_up
            self.short_profit = self.tp_down 
            
            if self.close_price >= self.long_stop:
                 self.cover(self.long_stop, abs(self.pos), True)
                
            elif self.close_price <= self.long_profit:
                 self.cover(self.long_profit, abs(self.pos), True)
            

        self.put_event()
        
    def on_order(self, order: OrderData):
        """
        Callback of new order data update.
        """
        pass

    def on_trade(self, trade: TradeData):
        """
        Callback of new trade data update.
        """
        self.put_event()

    def on_stop_order(self, stop_order: StopOrder):
        """
        Callback of stop order update.
        """
        pass


## 回測結果

In [14]:
from functools import partial

tickers = '2330.TSE'
symbol, exchange = tickers.split('.')
get_commission = partial(getCommission, symbol=symbol, exchange=exchange)
engine = BacktestingEngine()
engine.set_parameters(
    vt_symbol=tickers,
    interval=Interval.DAILY,
    start=datetime(2010, 12, 1),
    end=datetime(2019, 12, 1),
    rate=get_commission,
    slippage=getMinimumTick,
    size=1000,
    pricetick=getMinimumTick,
    capital=100_000,
    mode=BacktestingMode.BAR,
    collection_name='TSE')

boll_setting = {
    'boll_window' : 20,
    'boll_dev' : 2,
    'sl_multiplier' : 1.33,
    'tp_multiplier' : 1.5,
    'fixed_size' : 1.5,
}
engine.add_strategy(BollChannelStrategy, boll_setting)
engine.load_data()
engine.run_backtesting()
df = engine.calculate_result()
engine.calculate_statistics()
engine.show_chart()



2020-12-04 18:23:16.026252	开始加载历史数据
2020-12-04 18:23:16.026252	加载进度： [1%]
2020-12-04 18:23:16.027250	加载进度： [2%]
2020-12-04 18:23:16.027250	加载进度： [2%]
2020-12-04 18:23:16.027250	加载进度： [3%]
2020-12-04 18:23:16.027250	加载进度： [4%]
2020-12-04 18:23:16.027250	加载进度： [5%]
2020-12-04 18:23:16.027250	加载进度： [6%]
2020-12-04 18:23:16.027250	加载进度： [7%]
2020-12-04 18:23:16.027250	加载进度： [7%]
2020-12-04 18:23:16.027250	加载进度： [8%]
2020-12-04 18:23:16.027250	加载进度： [9%]
2020-12-04 18:23:16.027250	加载进度： [10%]
2020-12-04 18:23:16.027250	加载进度：# [11%]
2020-12-04 18:23:16.027250	加载进度：# [11%]
2020-12-04 18:23:16.027250	加载进度：# [12%]
2020-12-04 18:23:16.027250	加载进度：# [13%]
2020-12-04 18:23:16.027250	加载进度：# [14%]
2020-12-04 18:23:16.027250	加载进度：# [15%]
2020-12-04 18:23:16.027250	加载进度：# [16%]
2020-12-04 18:23:16.027250	加载进度：# [16%]
2020-12-04 18:23:16.027250	加载进度：# [17%]
2020-12-04 18:23:16.028245	加载进度：# [18%]
2020-12-04 18:23:16.028245	加载进度：# [19%]
2020-12-04 18:23:16.028245	加载进度：# [20%]
2020-12-04 18:23:16.028245	

## 勝賠率計算

In [15]:
average_win = df[df["trading_pnl"]>0]["trading_pnl"].mean()
average_loss = df[df["trading_pnl"]<0]["trading_pnl"].mean()
loss_rate = abs(average_loss)/(abs(average_win) + abs(average_loss))
loss_rate

0.4692833480102321